In [ ]:
import pandas as pd
import geopandas as gpd

import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo

# Set notebook mode to work in offline
pyo.init_notebook_mode()

### Goal - get and visualize available all Food Insecurity Classification data for South Sudan

### Step 1: Food Insecurity Data Point Extraction

In [ ]:
# Paste URL from https://fdw.fews.net/data-explorer/ipcFic?geographic_unit=SS
# NOTE `&fields=simple` parameter need to be removed to allow fnid field to be available

food_security_url = "https://fdw.fews.net/api/ipcphase/?format=json&country_code=SS&preference=best"
fic_dataframe = pd.read_json(food_security_url)
fic_dataframe.head()

In [ ]:
fic_dataframe.info()

### Step 2: Spatial Data Extraction 

In [ ]:
# Paste URL from https://fdw.fews.net/data-explorer/spatial?country_code=SS&unit_type=fsc_admin%2Cfsc_admin_lhz%2Cfsc_lhz%2Cfsc_rm_admin
spatial_url = "https://fdw.fews.net/api/feature/?format=geojson&country_code=SS&unit_type=fsc_admin&unit_type=fsc_admin_lhz&unit_type=fsc_lhz&unit_type=fsc_rm_admin"

spatial_dataframe = gpd.read_file(spatial_url)
spatial_dataframe.head()

In [ ]:
spatial_dataframe.info()

### Step 3: Merge Both DataFrames

In [ ]:
dataframe = spatial_dataframe[['fnid', 'unit_type', 'full_name', 'geometry']].merge(
    fic_dataframe, on='fnid', how='right'
)
# Remove not collected values
dataframe = dataframe[~dataframe['value'].isnull()].reset_index(drop=True)
dataframe.info()

### Step 4: Create a Map

In [ ]:
# Define necessary variables
unique_collection_dates = dataframe['reporting_date'].unique()
unique_collection_dates.sort()
recent_collection_date = unique_collection_dates[-1]
recent_dataframe = dataframe[dataframe['reporting_date'] == recent_collection_date]
bounds = dataframe.total_bounds
country_name = dataframe['country'].unique()
country_name = country_name[0] if len(country_name) == 1 else "various countries"

colorscheme = [
    [0, "#CDFACD"], 
    [0.25, "#FAE61E"], 
    [0.5, "#E67800"], 
    [0.75, "#C80000"], 
    [1, "#640000"]
]


In [ ]:
# Build a frame for each available collection date
frames = []

for collection_date in unique_collection_dates:
    # Filter the dataframe for the current date
    single_collection_dataframe = dataframe[dataframe['reporting_date'] == collection_date]

    # Create a frame for this date
    frames.append(
        go.Frame(
            data=[
                go.Choropleth(
                    name=str(collection_date),
                    ids=single_collection_dataframe['fnid'],
                    z=single_collection_dataframe['value'],
                    geojson=single_collection_dataframe.__geo_interface__,
                    locations=single_collection_dataframe.index,
                    colorscale=colorscheme,
                    colorbar=dict(
                        dtick=1, 
                        title=dict(text="IPC Phase")
                    ),
                    zmin=1,
                    zmax=5,
                    hovertemplate="<br>".join([
                        "%{customdata[0]}",
                        "IPC Phase %{customdata[1]}<extra></extra>"
                    ]),
                    customdata=single_collection_dataframe[["geographic_unit_full_name", 'value']].values
                )
            ],
            name=str(collection_date),
        )
    )

In [ ]:
# Build a map figure with time slider
fig = go.Figure(frames=frames)   

# Add time slider and buttons
fig.update_layout(
    margin={"t": 35, "b": 5, "l": 10, "r": 10},
    title_text=f"Acute Food Insecurity Phases in {country_name}",
    title_y=0.98,
    title_x=0.01,
    sliders=[
        dict(
            steps=[
                dict(
                    method="animate",
                    args=[
                        [str(collection_date)],
                        dict(frame=dict(duration=500, redraw=True), mode="immediate"),
                    ],
                    label=str(collection_date),
                )
                for collection_date in unique_collection_dates
            ],
            active=len(unique_collection_dates)-1,
            x=0.1,
            xanchor="left",
            y=0,
            yanchor="top",
        )
    ],
    updatemenus=[
        dict(
            type="buttons",
            showactive=True,
            x=0.1,
            y=0,
            xanchor="right",
            yanchor="top",
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,
                        dict(frame=dict(duration=1000, redraw=True), fromcurrent=True),
                    ],
                ),
                dict(label="Pause", method="animate", args=[[None], dict(mode="immediate")]),
            ],
        )
    ],
)

# Ensure the map zoomed to the data
fig.update_geos(
    visible=False,
    resolution=110,
    showcountries=True,
    countrycolor="#212721",
    showsubunits=True,
    showcoastlines=True,
    projection_type="mercator",
    fitbounds="locations",
    center={"lat": (bounds[1] + bounds[3]) / 2, "lon": (bounds[0] + bounds[2]) / 2},
    lonaxis_range=[bounds[0], bounds[2]],
    lataxis_range=[bounds[1], bounds[3]]
)

# Add initial trace
fig.add_trace(go.Choropleth(
    name=str(recent_collection_date),
    ids=recent_dataframe['fnid'],
    z=recent_dataframe['value'],
    geojson=recent_dataframe.__geo_interface__,
    locations=recent_dataframe.index,
    colorscale=colorscheme,
    colorbar=dict(
        dtick=1, 
        title=dict(text="IPC Phase")
    ),
    zmin=1,
    zmax=5,
    hovertemplate="<br>".join([
        "%{customdata[0]}",
        "IPC Phase %{customdata[1]}<extra></extra>"
    ]),
    customdata=recent_dataframe[["geographic_unit_full_name", 'value']].values
))